In [1]:
import networkx as nx
from pyvis.network import Network
import pandas as pd
import matplotlib.pyplot as plt
import textwrap

# Functions

wrapper = textwrap.TextWrapper(width=50) 

def create_vs_node(node, cr_verse):
    title = None 
    if node.startswith('Isaiah 7:14'):
        title = cr_verse[cr_verse['verse_title_ISH'] == node]['scripture_text_ISH'].values[0]
        title = wrapper.fill(text=title)
        title = title + '\nNOTE: Isaiah 7:14 is actually referenced 55 times,\nbut I limited to BoM verses with non-zero weights'
        color = "goldenrod"
    elif node.startswith('Isaiah'):
        title = cr_verse[cr_verse['verse_title_ISH'] == node]['scripture_text_ISH'].values[0]
        title = wrapper.fill(text=title)
        color = "blue"
    else:
        title = cr_verse[cr_verse['verse_title_BOM'] == node]['scripture_text_BOM'].values[0]
        title = wrapper.fill(text=title)
        color = "green"
    return(title, color)

def create_vs_edge(u,v, net_data):
    edge_color = {
        "Direct Quote": "black",
        "Shared Language": "dimgray",
        "Similar Theme": "silver"
    }.get(net_data['similarity_category'], "gray")
    edge_width = {
        "Direct Quote": 10,
        "Shared Language": 8,
        "Similar Theme": 6
    }.get(net_data['similarity_category'], 1)
    title = f'{net_data["Duhms_Class"]}-Isaiah \n {net_data["similarity_category"]} \nWeight: {net_data["similarity_score"]:.2f}'
    return(title, edge_width, edge_color)

def create_ch_node(ISH, BOM, node):
    if node.startswith('Isaiah'):
        # Extract Node Data
        data = ISH[ISH['combo_ISH'] == node].iloc[:, 3:11]
        cur_Duhm = data["Duhms_Class"].values[0]
        cur_word_count = data["word_count_ISH"].values[0]
        cur_verse_count = data["verse_count_ISH"].values[0]
        cur_bible_count = data["bible_term_in_ISH"].values[0]
        cur_ref_count = data["cross_ref_count"].values[0]
        cur_dq = data["Direct Quote"].values[0]
        cur_sl = data["Shared Language"].values[0]
        cur_st = data["Similar Theme"].values[0]
        # Make Title
        title = f'{cur_Duhm}-Isaiah \nWord Count: {cur_word_count} \nVerse Count: {cur_verse_count} \nVerses with Bible Terms: {cur_bible_count} \nCross Reference Count: {cur_ref_count} \n - Direct Quote: {cur_dq}  \n - Shared Language: {cur_sl} \n - Similar Theme: {cur_st}'
        color = "blue"
        return(title, color)

    else:
        # Extract Node Data
        data = BOM[BOM['combo_BOM'] == node].iloc[:, 2:10]
        cur_word_count = data["word_count_BOM"].values[0]
        cur_verse_count = data['verse_count_BOM'].values[0]
        cur_bible_count = data["bible_term_in_BOM"].values[0]
        cur_ref_count = data["cross_ref_count"].values[0]
        cur_dq = data["Direct Quote"].values[0]
        cur_sl = data["Shared Language"].values[0]
        cur_st = data["Similar Theme"].values[0]
        # Make Title
        title = f'Word Count: {cur_word_count}  \nVerse Count: {cur_verse_count} \nVerses with Bible Terms: {cur_bible_count} \nCross Reference Count: {cur_ref_count} \n - Direct Quote: {cur_dq} \n - Shared Language: {cur_sl} \n - Similar Theme: {cur_st}'
        color = "green"
        return(title, color)

def create_ch_edge(u,v,cr_group):
    data = cr_group[(cr_group['combo_BOM'] == u) & (cr_group['combo_ISH'] == v) | (cr_group['combo_BOM'] == v) & (cr_group['combo_ISH'] == u)]
    cur_verse_count = data['number_shared_verses'].values[0]
    cur_Duhm = data['Duhms_Class'].values[0]
    cur_dq = data["Direct Quote"].values[0]
    cur_sl = data["Shared Language"].values[0]
    cur_st = data["Similar Theme"].values[0]
    title = f'{cur_Duhm}-Isaiah \nCross Reference Count: {cur_verse_count} \n - Direct Quote: {cur_dq}  \n - Shared Language: {cur_sl} \n - Similar Theme: {cur_st}'
    width_scale = cur_verse_count/3

    if cur_dq > 0:
        edge_color = {
            "Proto": "lightskyblue",
            "Deutero": "crimson",
            "Trito": "crismon"
        }.get(data['Duhms_Class'].values[0], "gray")
    else:
        edge_color = 'gray'

    return(title, width_scale, edge_color)


# By Verse

In [2]:
cr = pd.read_csv('../CleanData/by_verse_cross_ref.csv', index_col=0)
cr_filtered = cr[(cr['word_count_BOM'] > 0) & (cr['word_count_ISH'] > 0) & ((cr['verse_title_ISH'] != 'Isaiah 7:14') | cr['similarity_score'] != 0)]

In [3]:
# Create a MultiGraph from data
G = nx.from_pandas_edgelist(cr_filtered,
                            source='verse_title_ISH',
                            target='verse_title_BOM',
                            edge_attr=['similarity_category', 'similarity_score', 'Duhms_Class'], 
                            create_using=nx.MultiGraph()
                            )

# Create a pyvis.Network object
net = Network(
    notebook=True,
    select_menu=True,
    cdn_resources='remote',
    height='690px',
    width='1450px'
)

# Editing node attributes
for node in G.nodes:
    title, color = create_vs_node(node, cr_filtered)
    net.add_node(node, color=color, title=title)

# Editing edge attributes
for u, v, data in G.edges(data=True):
    title, edge_width, edge_color = create_vs_edge(u,v,data)
    net.add_edge(u, v, title=title, width=edge_width, color=edge_color)



In [4]:

# Add custom options

net.set_options("""
{
  "nodes": {
    "borderWidth": 8,
    "borderWidthSelected": 24,
    "font": {
      "size": 26,
      "strokeWidth": 6
    },
    "scaling": {
      "min": 20,
      "max": 40
    }
  },
  "edges": {
    "scaling": {
      "min": 12,
      "max": 20
    },
    "selectionWidth": 3.5,
    "selfReferenceSize": null,
    "selfReference": {
      "size": 24,
      "angle": 0.7853981633974483
    },
    "smooth": false,
    "width": 14
  },
  "interaction": {
    "keyboard": {
      "enabled": true
    },
    "navigationButtons": true,
    "tooltipDelay": 100,
    "zoomSpeed": 0.7,
    "hover": true,
    "hoverConnected": true, 
    "hoverDegree": 1,
    "selectable": true,
    "selectConnectedEdges": true         
  },
  "physics": {
    "enabled": true,
    "barnesHut": {
      "gravitationalConstant": -3550,
      "centralGravity": 0.1,
      "springLength": 110,
      "avoidOverlap": 1
    },
    "minVelocity": 0.75
  }
} 
""")


# Show the graph
net.show('../Visuals/by_verse.html')


../Visuals/by_verse.html


# By Chapter

In [5]:
cr_group = pd.read_csv('../CleanData/by_chapter_cross_ref.csv',index_col=0)
ISH = pd.read_csv('../CleanData/ISH_summary.csv',index_col=0)
BOM = pd.read_csv('../CleanData/BOM_summary.csv',index_col=0)

In [6]:
G2 = nx.from_pandas_edgelist(cr_group, 
                            source='combo_ISH',
                            target='combo_BOM',
                            edge_attr=['Duhms_Class', 'number_shared_verses'],
                            create_using=nx.MultiDiGraph())

# Create a pyvis.Network object
net = Network(
    notebook=True,
    select_menu=True,
    cdn_resources='remote',
    height='690px',
    width='1450px'
)

# Editing node attributes
for node in G2.nodes:
    title, color = create_ch_node(ISH, BOM, node)
    net.add_node(node, color=color, title=title)

for u, v, data in G2.edges(data=True):
    title, width_scale, edge_color = create_ch_edge(u,v,cr_group)
    net.add_edge(u, v, title = title, width = width_scale, color = edge_color)



In [7]:

net.set_options(""" 
{
  "nodes": {
    "borderWidth": 1,
    "borderWidthSelected": 18,
    "font": {
      "size": 16,
      "strokeWidth": 3
    }
  },
  "edges": {
    "scaling": {
      "min": 12,
      "max": 20
    },
    "selectionWidth": 3.5,
    "selfReferenceSize": null,
    "selfReference": {
      "size": 24,
      "angle": 0.7853981633974483
    },
    "smooth": true,
    "width": 14
  },
  "interaction": {
    "keyboard": {
      "enabled": true
    },
    "navigationButtons": true,
    "tooltipDelay": 100,
    "zoomSpeed": 0.7,
    "hover": true,
    "hoverConnected": true, 
    "hoverDegree": 1,
    "selectable": true,
    "selectConnectedEdges": true         
  },
  "physics": {
    "repulsion": {
      "nodeDistance": 185,
      "damping": 0.06
    },
    "minVelocity": 0.75,
    "solver": "repulsion"
  }
}
""")


# Show the network visualization
net.show('../Visuals/by_chapter.html')

../Visuals/by_chapter.html
